Fractional Cover Annual Summary Workflow

1.Load daily fractional Cover for a year
2.Mask out areas where fractional cover is less than 20%
3.Use input data to convert -1 to NaNs
4.Calculate the mean of the resulting data. This will give the number of times the fractional cover was > 20% divided by the total number of observations.

In [4]:
import datacube
import matplotlib.pyplot as plt
from datacube import helpers
from datacube.utils import geometry
from scipy import ndimage
import xarray as  xr
import numpy as np

In [5]:
import sys
sys.path.append('/g/data/u46/users/sc0554/dea-notebooks/10_Scripts/')
import DEADataHandling

In [7]:
# dc = datacube.Datacube(app='lccs_loader')
dc=datacube.Datacube(config='/home/547/sc0554/datacube.conf')

In [8]:
# # Ayr
# x = (1500000, 1550000)
# y = (-2200000, -2150000)
# x = (1525000, 1550000)
# y = (-2225000, -2150000)

# # Diamentina
# x = (800000, 900000)
# y = (-2800000, -2700000)

# # Gwydir
# x = (1600000, 1700000)
# y = (-3400000, -3300000)

# # Leichhardt
# x = (850000, 900000)
# y = (-2000000, -1950000)

# # Kakadu
# x = (0, 100000)
# y = (-1350000, -1250000)

# # Hobart
# x = (1200000, 1300000)
# y = (-4800000, -4700000)

# # Perth
# x = (-1525000, -1475000)
# y = (-3625000, -3575000)
# x = (-1550000, -1450000)
# y = (-3650000, -3550000)

# # Murray Valley
# x = (1100000, 1200000)
# y = (-4000000, -3900000)

# Adelaide
# x = (570000, 590000)
# y = (-3850000, -383000)
# x = (550000, 650000)
# y = (-3850000, -3750000)

# Canberra
# x = ( 1543000, 1582000)
# y = ( -3982000, -3943000)
x = (1500000, 1600000)
y = (-4000000, -3900000)
# minx = 1500000
# maxx = 1600000
# miny = -4000000
# maxy = -3900000
# res_x = 25
# res_y = -25


# # Lake Eyre
# x = (550000, 600000)
# y = (-3000000, -2950000)

# # Blue mountains
# x = (1680000, 1750000)
# y = (-3850000, -3780000)

# # Australian Alps
# x = (1400000, 1500000)
# y = (-4100000, -4000000)

# # Collier Range
# x = (-1300000, -1200000)
# y = (-2700000, -2600000)

# # Coorong
# x = (600000, 700000)
# y = (-3950000, -3850000)

# # Brisbane
# x = (2000000, 2100000)
# y = (-3200000, -3100000)

# # Dundas
# x = (-1000000, -900000)
# y = (-3650000, -3550000)


#syd
# xs = 1679300
# ys = -4013201
# x = (xs-30000, xs+30000)
# y = (ys-30000, ys+30000)

product = 'fc'

query = {'time': ('2015-01-01', '2015-12-31')}
query['x'] = (x[0], x[1])
query['y'] = (y[0], y[1])
query['crs'] = 'EPSG:3577'

# query = {'time': ('2015-01-01', '2015-12-31')}
# query['x'] = (minx, maxx)
# query['y'] = (miny, maxy)
# query['crs'] = 'EPSG:3577'

In [9]:
mask_dict = {'cloud_acca': 'no_cloud',
             'cloud_fmask': 'no_cloud',
             'contiguous': True}

In [10]:
fc_data = DEADataHandling.load_clearlandsat(dc=dc, query=query, product=product,
                                  masked_prop=0.1,
                                  mask_dict=mask_dict,
                                  satellite_metadata=True)

Loading ls5
    Skipping ls5; no valid data for query
Loading ls7
    Ignoring SLC-off observations for ls7
    Skipping ls7; no valid data for query
Loading ls8
    Loading 37 filtered ls8 timesteps


MemoryError: 

In [ ]:
# fc_data = xr.open_dataset('netcdfs/canberrafc.nc')

In [ ]:
# fc_data2 = fc_data.drop(('satellite', 'data_perc', 'UE'))
# fc_data2.attrs['crs'] =  'test'
# fc_data2.NPV.attrs['crs'] = 'test'
# fc_data2.to_netcdf('perthfc.nc')

In [ ]:
# Create a mask to show areas where total vegetation is greater than the baresoil fraction of a pixel for each scene
tv_mask = fc_data['BS'] < (fc_data['PV']) #+ fc_data['NPV'])
tv = tv_mask.where(fc_data['PV'] > 0)

In [ ]:
num_obs = len(tv.time)

In [ ]:
# Calculate the proportion of time where total vegetatoin is greater than the bare soil fraction of a pixel for the input year
tv_summary = tv.mean(dim='time')

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(20,10))
plt.imshow(tv_summary)
plt.colorbar()

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(20,10))
tv_summary.plot.hist(bins=30, log=True)

In [ ]:
# The above histogram shows that most pixels in the summary have a greater total vegetation than baresoil for 100% of the year.

In [ ]:
if num_obs == 22: # maximum number of observations in a year
    threshold 

less observations mean each observation is representative of a smaller part of the year



In [ ]:
single = .167 / 22

threshold = .167 + (single*(22-num_obs)) # exponential decay e-num_obs fudge factor


In [ ]:
# Create a binary layer 

tv_summary_filt = tv_summary > .167
tv_summary_filt = tv_summary_filt*1
tv_summary_filt = tv_summary_filt.data.astype(float)

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(20,10))
plt.imshow(tv_summary_filt)
plt.colorbar()

In [ ]:
meta_d = fc_data.isel(time=0).drop('time')

In [ ]:
out.dims

In [ ]:
out = xr.Dataset({'fc_summary':(meta_d.dims,tv_summary)}, coords=meta_d.coords, attrs=meta_d.attrs)
helpers.write_geotiff('cbr_pvfcsummary.tif', out)